In [86]:
import os
import time
import random
import re
import pandas as pd
import numpy as np

In [80]:
import jieba

In [81]:
import pickle

In [82]:
def dump_pickle(obj,fname):
    with open(fname,'wb') as file:
        pickle.dump(obj,file)
def load_pickle(fname):
    return pickle.load(open(fname,'rb'))

In [83]:
claim_df = pd.read_pickle('../data/patent_data/processed/claim_df_seg_0630.pkl')

In [88]:
patent_text = claim_df.seg_text.tolist()
patent_text = [' '.join(i) for i in patent_text]

# 向量化 使用tfidf 重新训练 限定词表

In [85]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [89]:
patent_tfidf_obj = TfidfVectorizer(max_features=10000)

In [90]:
patent_tfidf_obj.fit(patent_text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [91]:
dump_pickle(patent_tfidf_obj,'../data/patent_data/processed/patent_tfidf_obj_1wdim_0701.pkl')

In [92]:
patent_all_tfidf_matrix = patent_tfidf_obj.transform(patent_text)

# 样例数据准备

In [93]:
sample_patent_data_tfidf = patent_tfidf_obj.transform(patent_text[:10000])

# 聚类 使用 k-means

# 关键词抽取做辅助

In [27]:
from textrank4zh import TextRank4Keyword,TextRank4Sentence

tr4w = TextRank4Keyword()
tr4w.analyze(patent_text[0],lower=True,window=2)

def topkword(text,topk=20):
    tr4w.analyze(text,lower=True,window=2)
    return tr4w.keywords[:topk]

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.773 seconds.
Prefix dict has been built succesfully.


# tfidf 抽取关键词

In [128]:
def topkword_tfidf(text,topk=20,tfidf_obj=patent_tfidf_obj):
    '''
    输入text为空格分隔的词们
    '''
    tfidf_vec = patent_tfidf_obj.transform([text])
    tfidf_vec = tfidf_vec.toarray()[0]
    wordlist = list(patent_tfidf_obj.vocabulary_.keys())
    order = np.argsort(tfidf_vec)[::-1]
    kwd = [wordlist[k] for k in order]
    pattern_none_chinese = re.compile('[0-9a-zA-Z]')
    kwd = list(filter(lambda x:not re.match(pattern_none_chinese,x),kwd))
    return kwd[:topk]

In [129]:
patent_text[0]

'一种 处理器 ， 包括 ： 位于 单个 芯片 封装 内 的 多个 处理 核 ， 其中 每个 处理 核 进一步 包括 ： 指令 取出 逻辑 单元 ， 用于 取出 一个 或 多个 指令 ； 指令 解码 逻辑 单元 ， 用于 解码 一个 或 多个 指令 ； 寄存器 文件 ， 包括 一组 向量 寄存器 ， 每个 向量 寄存器 存储 多个 向量 数据 元素 ； 执行 单元 ， 用于 执行 第一 指令 以 读取 单精度 浮点 值 ， 将 该值 转换 为 双 精度 浮点 值 ， 以及 将 结果 作为 可 变数 目的 N 个 向量 数据 元素 存储 在 寄存器 文件 的 向量 寄存器 中 ， 其中 所述 可变 数目 响应 于 所 执行 的 第一 指令 为 一个 或 多个 ； 其中 所述 执行 单元 用于 执行 第二 指令 以 将 单精度 浮点 值 转换 为 带符号 整数 值 并 将 结果 存储 在 储存 位置 ， 并且 其中 所述 执行 单元 用于 执行 第三 指令 以 将 单精度 浮点 值 转换 为 无 符号 整数 值 并 将 结果 存储 在 储存 位置 。 如权利要求 1 所述 的 处理器 ， 其特征在于 ， 所述 寄存器 文件 包括 一组 物理 寄存器 ， 用于 存储 浮点 值 和 向量 数据 元素 。'

In [131]:
topkword_tfidf(patent_text[2])

['脱硫',
 '速比',
 '耦合',
 '近似',
 '那个',
 '络合物',
 '照相机',
 '偏置',
 '倾斜地',
 '端面',
 '基站',
 '图像编码',
 '波束',
 '灭火',
 '图像处理',
 '炎症',
 '圆环',
 '绝缘体',
 '收纳',
 '还原性']

In [ ]:
with open('../data/patent_data/processed/claim_kword_tfidf.txt','w',encoding='utf-8') as file:
    for num,_ in enumerate(claim_df.itertuples()):
        if num % 1000==0:
            print('[{0}] Num:{1}'.format(time.ctime(),num))
        kw = topkword_tfidf(' '.join(claim_df.loc[num,'seg_text']))
        kw = '|'.join(kw)
        file.write(kw+'\n')

[Tue Jul  2 09:47:47 2019] Num:0
[Tue Jul  2 09:48:03 2019] Num:1000
[Tue Jul  2 09:48:18 2019] Num:2000
[Tue Jul  2 09:48:33 2019] Num:3000
[Tue Jul  2 09:48:48 2019] Num:4000
[Tue Jul  2 09:49:03 2019] Num:5000
[Tue Jul  2 09:49:19 2019] Num:6000
[Tue Jul  2 09:49:34 2019] Num:7000
[Tue Jul  2 09:49:49 2019] Num:8000
[Tue Jul  2 09:50:04 2019] Num:9000
[Tue Jul  2 09:50:19 2019] Num:10000
[Tue Jul  2 09:50:35 2019] Num:11000
[Tue Jul  2 09:50:50 2019] Num:12000
[Tue Jul  2 09:51:05 2019] Num:13000
[Tue Jul  2 09:51:20 2019] Num:14000
[Tue Jul  2 09:51:35 2019] Num:15000
[Tue Jul  2 09:51:50 2019] Num:16000
[Tue Jul  2 09:52:06 2019] Num:17000
[Tue Jul  2 09:52:21 2019] Num:18000
[Tue Jul  2 09:52:36 2019] Num:19000
[Tue Jul  2 09:52:52 2019] Num:20000
[Tue Jul  2 09:53:07 2019] Num:21000
[Tue Jul  2 09:53:22 2019] Num:22000
[Tue Jul  2 09:53:37 2019] Num:23000
[Tue Jul  2 09:53:52 2019] Num:24000
[Tue Jul  2 09:54:07 2019] Num:25000
[Tue Jul  2 09:54:23 2019] Num:26000
[Tue Jul  2 09